Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [312]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from collections import Counter
import spacy
nlp = spacy.load("en_core_web_lg")

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train.head()
train = train.set_index('id')
test = test.set_index('id')
print(train.shape, test.shape)

(4087, 2) (1022, 1)


In [128]:
test

,description
id,
3461,\nStyle: Speyside single malt scotch Color: Wa...
2604,"\nVery bright and lively, with a nice balance ..."
3341,\nA new oloroso-forward Chivas positioned to s...
3764,\nAged in bourbon casks and then enhanced in R...
2306,\nThere is a freshness to the wood on the nose...
...,...
2853,"\nCare for a small batch, bourbon-matured blen..."
219,"\nThis is the pick of the bunch, the whisky eq..."
1286,"\nPort Ellen, for sure! Very old-fashioned in ..."


In [2]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [51]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, 
                              test_size=0.2,
                              stratify=train['ratingCategory'])
print(train.shape, val.shape)

(3269, 2) (818, 2)


In [353]:
#addded before class
import tqdm
target  = 'ratingCategory'
y_train = train[target]
y_val   = val[target]

X_train = train.drop(columns=[target])
X_val   = val.drop(columns=[target])
# X_train = get_word_vectors(X_train['description'])
# X_val   = get_word_vectors(X_val['description'])
# X_train = X_train['description'].apply(my_tokenizer)
# X_val = X_val['description'].apply(my_tokenizer)

In [348]:
y_val   = val[target]

In [349]:
X_train.shape, y_train.shape

id
551     1
400     1
4533    0
163     1
2511    1
       ..
4335    0
4781    0
4661    0
4368    0
3260    1
Name: ratingCategory, Length: 818, dtype: int64

In [59]:
X_train.head()

,description
id,
182,\nThe sixth and last bottling from the Legacy ...
2842,"\nA terrific value blend of Macallan, Glenroth..."
2621,\nThe latest incarnation of Springbank’s Longr...
4751,\nWhy is it that so many great beer-producing ...
1326,\nTop notes of spearmint and green apple part ...


In [60]:
train.ratingCategory.value_counts(normalize=True)

1    0.704803
0    0.279290
2    0.015907
Name: ratingCategory, dtype: float64

In [61]:
val.ratingCategory.value_counts(normalize=True)

1    0.705379
0    0.278729
2    0.015892
Name: ratingCategory, dtype: float64

### Define Pipeline Components

In [136]:
import re

# Optional: Use a custom tokenizer
def my_tokenizer(text):
    clean_text = re.sub('[^a-zA-Z ]', '', text)
    tokens = clean_text.lower().split()
    return tokens
def get_word_vectors_avg(docs):
    out = []
    for doc in docs:
        vect = nlp(doc).vector
        out.append(sum(vect)/len(vect))
    return out
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [324]:
def get_adj(docs):
    out = []
    for doc in docs:
        adj = ""
        doc = nlp(doc)
        for token in doc:
            if token.pos_ == 'ADJ':
                adj += " " + token.text
        out.append(adj)
    return out

In [358]:
X_train['adj'] = get_adj(X_train['description'])

In [330]:
adj_count = count(X_train)


In [359]:
X_val['adj'] = get_adj(X_val['description'])

In [310]:
def count(tokens):
    """
    Calculates some basic statistics about tokens in our corpus (i.e. corpus means collections text data)
    """
    # stores the count of each token

    word_counts = Counter()

    # stores the number of docs that each token appears in 
    appears_in = Counter()
    total_docs = len(tokens)

    for token in tokens:
        # stores count of every appearance of a token 
        word_counts.update(token)
        # use set() in order to not count duplicates, thereby count the num of docs that each token appears in
        appears_in.update(set(token))

    # build word count dataframe
    temp = zip(word_counts.keys(), word_counts.values())
    wc = pd.DataFrame(temp, columns = ['word', 'count'])

    # rank the the word counts
    wc['rank'] = wc['count'].rank(method='first', ascending=False)
    total = wc['count'].sum()

    # calculate the percent total of each token
    wc['pct_total'] = wc['count'].apply(lambda token_count: token_count / total)

    # calculate the cumulative percent total of word counts 
    wc = wc.sort_values(by='rank')
    wc['cul_pct_total'] = wc['pct_total'].cumsum()


    # create dataframe for document stats
    t2 = zip(appears_in.keys(), appears_in.values())
    ac = pd.DataFrame(t2, columns=['word', 'appears_in'])

    # merge word count stats with doc stats
    wc = ac.merge(wc, on='word')

    wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)

    return wc.sort_values(by='rank')

In [120]:
# ADDED BEFORE CLASS
from sklearn.svm import LinearSVC
from sklearn.decomposition import TruncatedSVD
#clf = RandomForestClassifier()
vect = TfidfVectorizer(stop_words='english',
                       ngram_range=(1,2),
                       min_df =2,
                       max_df=.5
                       )
svm = LinearSVC()
svd = TruncatedSVD(n_components=2,
                   algorithm='randomized',
                   n_iter=10)

rfc = LogisticRegression(solver='lbfgs')

pipe = Pipeline([
    ('vect', vect), 
    ('clf', svm)]
)
lsa = Pipeline([('vect', vect), ('svd', svd)])

pipe_2 = Pipeline([
    ('lsa', lsa),
    ('clf', rfc)
])

In [354]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train)
classifier.score(X_val, y_val)

ValueError: could not convert string to float: '\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [285]:
def get_entities(X):
    descriptions = X['description']
    entity_list = []

    for description in descriptions:
        doc = nlp(description)
        entitys = ""
        for entity in doc.ents:
            entitys += " " + entity.text
#         for chunk in doc.noun_chunks:
#             chunks += " " + chunk.text
        entity_list.append(entitys)
#         entity_list.append(chunks)
    return entity_list

In [286]:
val_entities = get_entities(X_val)


In [287]:
entity_list = get_entities(X_train)

In [291]:
X_train['entities'] = entity_lis
X_val['entities']   = val_entities

In [296]:
X_train.head()

,description,entities
id,,
1321,"\nSometimes, when whisky is batched, a few lef...",Canadian Club the 1970s Granny Smith white ce...
3861,\nAn uncommon exclusive bottling of a 6 year o...,6 year old Wyoming
655,\nThis release is a port version of Amrut’s In...,Amrut Intermediate Sherry a few months damson
555,\nThis 41 year old single cask was aged in a s...,This 41 year old American 1970s
1965,"\nQuite herbal on the nose, with aromas of dri...",


In [360]:
X_train = get_word_vectors(X_train['adj'])
X_val   = get_word_vectors(X_val['adj'])

In [351]:
y_train

id
1321    1
3861    0
655     1
555     1
1965    1
       ..
3342    1
3130    1
2811    1
478     1
1209    1
Name: ratingCategory, Length: 4087, dtype: int64

In [361]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_val_pred = classifier.predict(X_val)
accuracy_score(y_val, y_val_pred)

0.9987775061124694

In [355]:
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2,
                       max_df=0.7)

svd = TruncatedSVD(n_components=2,
                  algorithm='randomized',
                  n_iter=10)

svm = LinearSVC(C=0.5, 
                penalty='l2')

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # LSI (SVD Dim Reduction)
    ('clf', svm)         # LinearSVC Classifier
])
pipe.fit(X_train, y_train)

ValueError: max_df corresponds to < documents than min_df

In [334]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
y_val_pred = pipe.predict(X_val['adj'])
accuracy_score(y_val, y_val_pred)

0.7053789731051344

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [162]:
classifier

RandomForestClassifier()

In [176]:
# # Added Before Class
# # Small grid search to reduce runtime
parameters = {
    'n_estimators': (50, 100, 200, 300, 400),
    'class_weight':['balanced', None]
}


grid_search = GridSearchCV(classifier, parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'class_weight': ['balanced', None],
                         'n_estimators': (50, 100, 200, 300, 400)},
             verbose=1)

In [177]:
grid_search.best_score_

0.7359906988714751

In [257]:
grid_search.best_params_

{'clf__C': 2.0,
 'clf__penalty': 'l2',
 'svd__n_components': 500,
 'vect__max_features': 20000,
 'vect__ngram_range': (1, 3)}

In [184]:
classifier = RandomForestClassifier(**grid_search.best_params_)
classifier.fit(X_train, y_train)
y_val = classifier.predict(X_val)
accuracy_score(X_val, y_val)

1.0

In [124]:
parameters = {
    'lsa__vect__min_df': (2, 5),
    'lsa__vect__max_df': (.3, .7),
}


grid_search = GridSearchCV(pipe_2, parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'lower'

### Evaluate on Validation Data

In [269]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
y_val = classifier.predict(X_val)
accuracy_score(val.ratingCategory, y_val)

ValueError: could not convert string to float: '\nA mix of 29 bourbon and sherry casks that were mingled for 3 months in Balvenie’s bespoke marrying vessel, the Tun. Fragrant oriental spices on the nose: ginger, cinnamon, fresh orange juice, and brittle toffee. Full and smooth on the palate, with sweet sherry and spicy Jaffa orange. Long in the finish, with dark chocolate, oak tannins, raisins, and licorice at the close.'

In [211]:
from sklearn.metrics import classification_report

print(classification_report(val.ratingCategory, y_val))

              precision    recall  f1-score   support

           0       0.95      0.62      0.75       228
           1       0.87      0.99      0.93       577
           2       1.00      0.85      0.92        13

    accuracy                           0.89       818
   macro avg       0.94      0.82      0.86       818
weighted avg       0.89      0.89      0.88       818



### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [212]:
# Predictions on test sample
X_test = get_word_vectors(test['description'])
# pred = grid_search.predict(X_test)

In [213]:
pred = classifier.predict(X_test)

In [214]:
submission = pd.DataFrame({'id': test.index, 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [187]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [215]:
subNumber = 5

In [253]:
entity_list

[' Canadian Club the 1970s Granny Smith white cedar Canadian Club Australia 133',
 ' 6 year old Wyoming',
 ' Amrut Intermediate Sherry a few months damson',
 ' This 41 year old American 1970s',
 '',
 ' Cooley Irish Irish',
 ' This year’s Ardbeg Day World Cup Brazil Auri Verdes (Green Ardbeg',
 ' Talisker Diageo 25 and 30 year old 45.8% 300',
 '',
 ' Seville first Bowmore Bowmore 30+ years']

In [216]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [282]:
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2,
                       max_df=0.7)

svd = TruncatedSVD(n_components=2,
                  algorithm='randomized',
                  n_iter=10)

svm = LinearSVC(C=0.5, 
                penalty='l2')

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # LSI (SVD Dim Reduction)
    ('clf', svm)         # LinearSVC Classifier
])
pipe.fit(entity_list, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [8174, 4087]

### (Skip this Section for now) Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [198]:
parameters = {
    'vect__max_features': (10000, 20000),
    'vect__ngram_range':[(1,2), (1,3)],
    'clf__penalty': ('l1','l2'),
    'clf__C': (0.1, 0.5, 1., 2.),
    'svd__n_components': (100, 500),
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=10)
grid_search.fit(X_train['description'], y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan 0.72767295 0.72840765 0.72326898 0.72473747
 0.73305821 0.73330331 0.73110132 0.73036783        nan        nan
        nan        nan        nan        nan        nan        nan
 0.74040155 0.74211245 0.74137895 0.74088905 0.75092009 0.75214498
 0.75018599 0.75238828        nan        nan        nan        nan
        nan        nan        nan        nan 0.74260444 0.74529452
 0.74333614 0.74358273 0.75214318 0.75238918 0.75434427 0.75458996
        nan        nan        nan        nan        nan        nan
        nan        nan 0.74627162 0.74382454 0.74382573 0.74407023
 0.75165718 0.75336837 0.75263158 0.75581485]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(max_df=0.3, min_df=2,
                                                        ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('svd', TruncatedSVD(n_iter=10)),
                                       ('clf', LinearSVC(C=0.5))]),
             n_jobs=4,
             param_grid={'clf__C': (0.1, 0.5, 1.0, 2.0),
                         'clf__penalty': ('l1', 'l2'),
                         'svd__n_components': (100, 500),
                         'vect__max_features': (10000, 20000),
                         'vect__ngram_range': [(1, 2), (1, 3)]},
             verbose=10)

In [199]:
grid_search.best_score_

0.7558148512807008

### Evaluate on Validation Data

In [280]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
y_val_pred = pipe.predict(val_entities)
accuracy_score(y_val, y_val_pred)

ValueError: Found input variables with inconsistent numbers of samples: [818, 1636]

In [201]:
from sklearn.metrics import classification_report

print(classification_report(val.ratingCategory, y_val))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       228
           1       1.00      1.00      1.00       577
           2       1.00      1.00      1.00        13

    accuracy                           1.00       818
   macro avg       1.00      1.00      1.00       818
weighted avg       1.00      1.00      1.00       818



### Make a Submission File

In [363]:
# Predictions on test sample
# pred = pipe.predict(test['description'])
X_test = get_adj(test.description)
X_test = get_word_vectors(X_test)
pred = classifier.predict(X_test)

In [364]:
submission = pd.DataFrame({'id': test.index, 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [365]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [366]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber += 1
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?